In [1]:
import nltk

def sen_seg(data): 
    sens = []
    to_replace = ['et al. ', 'Fig. ', 'e.g. ', 'i.e. ', 'Ref. ', 'Figs. ', ' ca. ', 'approx. ', '(ca. ', 'etc.) ']
    for tr in to_replace:
        data = data.replace(tr, tr[:-2]+'####@')
    tmp = nltk.sent_tokenize(data)
    # tmp = data.split('. ')
    for i, t in enumerate(tmp):
        for tr in to_replace:
            t = t.replace(tr[:-2]+'####@', tr)
        tmp[i] = t
    for t in tmp:
        split_t = nltk.word_tokenize(t)
        if 10<len(split_t)<90:
            sens.append(split_t)
    return sens

In [2]:
import kashgari
from kashgari.tasks.classification import CNN_Attention_Model, CNN_Model

load_CNN = CNN_Model.load_model('CNN_94_95_93')
load_CNN_Attention = CNN_Attention_Model.load_model('CNN_Attention_91')

2022-08-29 14:23:03,001 [DEBUG] kashgari - ------------------------------------------------
2022-08-29 14:23:03,004 [DEBUG] kashgari - Loaded gensim word2vec model's vocab
2022-08-29 14:23:03,004 [DEBUG] kashgari - model        : mat_embedding3/vectors.txt
2022-08-29 14:23:03,005 [DEBUG] kashgari - word count   : 529690
2022-08-29 14:23:03,005 [DEBUG] kashgari - Top 50 words : ['the', 'of', '.', ',', 'and', '<nUm>', 'in', 'a', 'to', ')', '(', 'with', '-', 'for', 'is', 'by', 'on', 'was', 'at', 'were', 'that', '–', 'as', 'are', 'from', '/', 'an', 'temperature', 'surface', 'using', 'high', 'which', 'C', '°', 'this', '%', 'In', 'it', 'A', '=', 'structure', 'properties', ':', 'phase', 'results', 'effect', 'these', 'than', 'based', 'different']
2022-08-29 14:23:03,005 [DEBUG] kashgari - ------------------------------------------------
2022-08-29 14:24:46,446 [DEBUG] kashgari - ------------------------------------------------
2022-08-29 14:24:46,448 [DEBUG] kashgari - Loaded gensim word2vec m

In [3]:
import json

with open('total_doi_year.json', 'r', encoding='utf-8') as f:
    doi_year = json.load(f)

In [4]:
def get_opinions(sens):
    if sens == []:
        return [], []
    find_opinion = []
    result = load_CNN.predict(sens)
    for i, tx in enumerate(sens):
        if result[i] == 'opinion':
            find_opinion.append(tx)
    if find_opinion == []:
        return [], []
    find_opps = []
    find_chas = []
    result = load_CNN_Attention.predict(find_opinion)
    for i, tx in enumerate(find_opinion):
        if result[i] == 'driver':
            find_opps.append(' '.join(tx))
        else:
            find_chas.append(' '.join(tx))
    return find_opps, find_chas

In [5]:
def ALD_justify(text):
    judge_words = ['ALD', 'Atomic Layer Deposition']
    flag = 0
    for j in judge_words:
        if j in text:
            flag = 1
    if flag == 1:
        return True
    else:
        return False

## Elsevier

In [6]:
sen_dict = {}

In [9]:
import os
from tqdm import tqdm

elsevier = os.listdir('Elsevier2/')
for e in tqdm(elsevier):
    with open('Elsevier2/'+e, 'r', encoding='utf-8') as f:
        data = f.readlines()
        doi = data[0][:-1]
        if doi not in doi_year.keys():
            print(doi)
            continue
        start_words = ['Graphical abstract','Abstract', 'Corresponding author', 'Correspondence to:', 'Introduction', 'Keywords']
        s_find = 0
        for s in start_words:
            if s in data[1]:
                start = data[1].find(s)
                s_find = 1
        if s_find == 0:
            start = 0
        if 'Reference' in data[1]:
            end = data[1].rfind('Reference')
        else:
            end = len(data[1])
        fulltext = data[1][start:end]
        ald_or_not = ALD_justify(fulltext)
        opps, chas = get_opinions(sen_seg(fulltext))
        for o in opps:
            sen_dict[o] = {}
            sen_dict[o]['ald'] = ald_or_not
            sen_dict[o]['doi'] = doi
            sen_dict[o]['year'] = doi_year[doi]
            sen_dict[o]['senti'] = 'o'
        for c in chas:
            sen_dict[c] = {}
            sen_dict[c]['ald'] = ald_or_not
            sen_dict[c]['doi'] = doi
            sen_dict[c]['year'] = doi_year[doi]
            sen_dict[c]['senti'] = 'c'

 25%|██▍       | 21690/87300 [1:01:21<2:11:07,  8.34it/s]

10.1016/j.joule.2018.07.020


100%|██████████| 87300/87300 [3:33:06<00:00,  6.83it/s]   


In [10]:
json_str = json.dumps(sen_dict, indent=4)
with open('sen_dict.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)    

## RCS

In [15]:
with open('sen_dict.json', 'r', encoding='utf-8') as f:
    sen_dict = json.load(f)

In [16]:
print(len(sen_dict.keys()))

1582831


In [17]:
import os
from chemdataextractor import Document
from chemdataextractor.scrape import Selector
from chemdataextractor.scrape.pub.rsc import RscHtmlDocument
from chemdataextractor.doc import Document, Title, Heading, Paragraph, Citation, Table, Figure, Caption, Footnote
from chemdataextractor.reader.rsc import RscHtmlReader
from tqdm import tqdm


filepath = 'RSC/'
filename = os.listdir(filepath)

all_text = []
for fn in tqdm(filename):
    f = open(filepath+fn, 'rb')
    try:
        doc = Document.from_file(f, readers=[RscHtmlReader()])
    except:
        print(fn+'cannot work')
        continue
    fulltext = ''
    for e in doc.elements:
        if type(e) == Title:
            str_e = str(e).replace('\n', '')
            fulltext += str_e+'.'
        if type(e) == Paragraph:
            word_num = len(str(e).split(' '))
            if word_num > 100:
                str_e = str(e).replace('\n', '')
                fulltext += ' '+str_e
    htmlstring = open(filepath+fn, 'rb').read()
    sel = Selector.from_text(htmlstring)
    scrape = RscHtmlDocument(sel)
    doi = scrape.serialize()['doi']
    if doi not in doi_year.keys():
        doi = doi.lower()
        if doi not in doi_year.keys():
            doi = doi.upper()
            if doi not in doi_year.keys():
                print('doi not work')
                continue
    ald_or_not = ALD_justify(fulltext)
    opps, chas = get_opinions(sen_seg(fulltext))
    for o in opps:
        sen_dict[o] = {}
        sen_dict[o]['ald'] = ald_or_not
        sen_dict[o]['doi'] = doi
        sen_dict[o]['year'] = doi_year[doi]
        sen_dict[o]['senti'] = 'o'
    for c in chas:
        sen_dict[c] = {}
        sen_dict[c]['ald'] = ald_or_not
        sen_dict[c]['doi'] = doi
        sen_dict[c]['year'] = doi_year[doi]
        sen_dict[c]['senti'] = 'c'

  4%|▍         | 380/9399 [03:19<1:11:15,  2.11it/s]

RCS_10338.htmlcannot work


  5%|▍         | 465/9399 [04:05<1:23:56,  1.77it/s]

RCS_10415.htmlcannot work


 37%|███▋      | 3462/9399 [28:18<41:09,  2.40it/s]  

...

 91%|█████████ | 8510/9399 [1:11:44<10:15,  1.44it/s]

RCS_18571.htmlcannot work


 91%|█████████ | 8561/9399 [1:12:17<06:46,  2.06it/s]

RCS_18617.htmlcannot work


100%|██████████| 9399/9399 [1:20:10<00:00,  1.95it/s]


In [18]:
print(len(sen_dict.keys()))

1748100


In [19]:
json_str = json.dumps(sen_dict, indent=4)
with open('sen_dict.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str) 

## Springer

In [15]:
with open('sen_dict.json', 'r', encoding='utf-8') as f:
    sen_dict = json.load(f)
print(len(sen_dict.keys()))

1807886


In [17]:
from tqdm import tqdm
import os

doi_dic = {}
springer = os.listdir('springer_fulltexts/')
for s in tqdm(springer):
    with open('springer_fulltexts/'+s, 'r', encoding='utf-8') as f:
        data = f.readlines()
        fulltext = ''
        for d in data[1:-1]:
            fulltext+=d[:-1]+' '
        fulltext += data[-1][:-1]
        doi = data[0][:-1]
        if doi not in doi_year.keys():
            continue
        ald_or_not = ALD_justify(fulltext)
        opps, chas = get_opinions(sen_seg(fulltext))
        for o in opps:
            sen_dict[o] = {}
            sen_dict[o]['ald'] = ald_or_not
            sen_dict[o]['doi'] = doi
            sen_dict[o]['year'] = doi_year[doi]
            sen_dict[o]['senti'] = 'o'
        for c in chas:
            sen_dict[c] = {}
            sen_dict[c]['ald'] = ald_or_not
            sen_dict[c]['doi'] = doi
            sen_dict[c]['year'] = doi_year[doi]
            sen_dict[c]['senti'] = 'c'

100%|██████████| 6433/6433 [17:44<00:00,  6.04it/s]


In [18]:
print(len(sen_dict.keys()))

1823056


In [19]:
json_str = json.dumps(sen_dict, indent=4)
with open('sen_dict.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str) 

## solar tile

In [21]:
with open('tile_dict.json', 'r', encoding='utf-8') as f:
    tile_dict = json.load(f)

for t in tile_dict.keys():
    if 'fulltext' in tile_dict[t].keys():
        fulltext = tile_dict[t]['fulltext']
    else:
        fulltext = tile_dict[t]['ab']
    opps, chas = get_opinions(sen_seg(fulltext))
    if opps != [] or chas != []:
        print(t)
        if 'doi' in tile_dict[t].keys():
            print(tile_dict[t]['doi'])
        for o in opps:
            print(o)
        for c in chas:
            print(c)
        print('\n')

Numerical simulation of a novel pavement integrated photovoltaic thermal (PIPVT) module
10.1016/j.apenergy.2020.116287
Thereby , pavements are quite suitable for solar energy utilization as energy , either electricity or heat , which can be generated with few cars or pedestrians shadowing the surface when solar radiation is strong .
Meanwhile , pavements also have enormous potential to generate electricity .
Besides , PV pavement was also found effective for heat island mitigation , decreasing the surface temperature and ambient temperature by 5 °C and about 1 °C , respectively [ 19 ] .
Additionally , high module temperature will also accelerate the rate of aging [ 21 ] .
Thereby , necessary measures should be taken to extract the surplus heat timely and keep the PV pavement in relatively low temperature .
Since there have been a lot of research and projects about the PV pavement confirming its mechanical feasibility , the PIPVT , another similar technology with the combination of PVT 

Thermal impact of adhesive-mounted rooftop PV on underlying roof shingles
10.1016/j.solener.2018.09.079
To realize this huge market potential , however , the non-hardware ( soft ) material and installation costs must be reduced ( Ardani et al. , 2013 ; Barbose et al. , 2013 ; Morris et al. , 2013 ; Morris et al. , 2014 ) .
To ensure the sufficient structural integrity , we studied the wind uplift , creep performance and durability of the adhesives ( Honeker et al. , 2016 ) .
In addition , since a typical roof accounts for about 14 % of the total cooling loads in a residential building ( Buildings Energy Databook , 2010 ) , the potential for heat build-up caused by adhesive-mounted rooftop PV may increase the cooling demand of the building .
It was concluded that an appropriate air gap could reduce the heat load of the building by at least 1.85 kWh/m2 per year .
Thermal isolation reduces the influence of the junction potential caused by the dissimilarity of the TC wire and the wire lead

Data on roof renovation and photovoltaic energy production including energy storage in existing residential buildings
10.1016/j.dib.2022.107874
Keywords Roof insulation Energy efficiency PV systems Residential building retrofit Building energy simulations Specification Table Subject Energy .


Design and development of energy efficient re-roofing solutions
10.1016/j.renene.2019.11.128
Keywords Energy-efficient building materials Retrofit strategies BIM Solar panels 1 Introduction In order to develop a more holistic method for energy efficient refurbishment of buildings , it is necessary to focus on decision-making processes , particularly with regard to building envelope components .
Retrofit and refurbishment strategies can result in energy efficiency improvements and cost reduction initiatives for buildings .
The proposed methodology supports development of sustainable energy retrofitting strategies for the existing roofs .
Moreover , the study indicated that appropriate insulation c

Increasing the transmission performance of a conventional 110 kV cable line by combining a hydronic concrete pavement system with photovoltaic
10.1007/s00202-020-01167-4
In particular , the PV cells would generate waste heat and direct current electricity from sunlight ( using the PV effect ) , the cable bedding would conduct waste heat well from the cables to the heat collecting pipes of the HCP system ( by improved heat conduction ) , while the HCP system would collect and store waste heat from the PV cells and cables as well as some heat from the Sun ( using the heat exchanger principle and heat storage capability ) .


A Novel Control System for Solar Tile Micro-Inverters
Current sensors are usually required in the micro-inverter circuitry in order to perform maximum power point tracking ( MPPT ) , as well as in the current loop to shape the output current to a sinusoidal waveform synchronous with the utility grid .
The proposed method in this paper is able to estimate the current 

An experimental study of the impact of cool roof on solar PV electricity generations on building rooftops in Sharjah, UAE
10.1093/ijlct/ctz008
The preliminary findings of the experimental study indicated that there is a likely impact of 5-10 % improvement of electricity generation with the cool roof applications .


Green and cool roof choices integrated into rooftop solar energy modelling
10.1016/j.apenergy.2021.117082
Due to the reduction in surface temperature , these sustainable roofs have the potential to increase the yield of rooftop photovoltaic ( PV ) panels , whose conversion efficiency is temperature dependent [ 10 ] .
As summarized in review studies ( e.g. , [ 13,14 ] ) , results from experimental studies are promising , especially in warmer regions ( e.g. , Hui & Chan ( 2011 ) [ 15 ] , Chemisana & Lamnatou ( 2014 ) [ 16 ] , Osma-Pinto & Ordoñez-Plata ( 2019 ) [ 17 ] ) , supporting the need for integrated PV-sustainable roof evaluation methods .
We argue that an accurate int

...

Performance of shingled solar modules under partial shading
10.1002/pip.3486
This makes shingling a very attractive approach for solar cell interconnection .


Operation and performance of the ATLAS Tile Calorimeter in Run 1
10.1140/epjc/s10052-018-6374-z
The energy and time calibration methods performed excellently , resulting in good stability of the calorimeter response under varying conditions during the LHC Run 1 .
The results demonstrate excellent performance in accord with specifications mentioned in the Technical Design Report .


Use of microspheres in thermally insulating hybrid solar panels
10.1016/j.egypro.2018.08.090
The cover glass used in classical solar panel has been replaced by a transparent resin which provides comparable optical properties but also improves the mechanical features and allows easier cleaning and maintenance .
The cover glass used in classical solar panel has been replaced by a transparent resin which provides comparable optical properties but also im

Multi-parameter heat transfer analysis of molten PCM in an inclined enclosure
10.1016/j.applthermaleng.2022.118209
The photovoltaic ( PV ) system seems to be a good remedy to this dilemma , which can convert solar energy into electrical energy with no pollution and low operating cost [ 1,2 ] .
There is considerable evidence indicating that PV/PCM system can significantly reduce the temperature of PV panels and enhance the generation efficiency .
Despite heat transfer can be enhanced , non-uniformity of solid–liquid interface and temperature distribution occurred during the PCM melting process , which could prolong the total PCM melting time .
Another technique that helps to reduce the temperature of PV panels and reach a higher electrical efficiency is the forced air circulation , with the advantage of being economically feasible [ 25 ] .
Hence , the air-based PV/PCM system could reduce panel temperature and improve indoor air for thermal comfort .
Higher Reynolds number of the convect

Design and Optimization of a Magnetic Levitation Load Reduction Device for a Hydropower Unit
10.1109/ACCESS.2021.3060544
To save cost , the capacity of a single unit has been greatly increased , and many higher technical requirements have been presented for hydropower units , among which one of the key technologies is the thrust bearing .
Therefore , the thrust bearing needs to be designed to improve the bearing capacity .
Instead of improving the traditional mechanical structure and using new materials , this paper proposes a levitation load reduction device for the thrust bearing to reduce the pv value of the kinetic energy on the thrust bearing .


Development of MoO3/Au/MoO3 Top Transparent Conducting Electrode for Organic Solar Cells on Opaque Substrates
10.1002/ente.202100689
For realization of flexible solar cells on cheap opaque substrates such as steel foils , it is vital to develop new top transparent conducting electrodes as alternatives to indium tin oxide .
Improvement in 

10.1007/978-981-15-7827-4_18
This work can lay a path to development of lightweight roofing and supporting structures like thin shells , silos , etc .


Analysis of the outer divertor hot spot activity in the protection video camera recordings at JET
10.1016/j.fusengdes.2018.12.079
This issue has become especially important after the installation of the ITER-like wall ( ILW ) [ 2 ] , where the carbon-fibre composite ( CFC ) tiles in the divertor were replaced with bulk tungsten ( W ) and W-coated CFC tiles which impose stricter heat load restrictions .
To reduce the frequency of false positive alarms from the thermal protection system , recent efforts have been invested in the characterisation of appearance , and cataloguing hotspots in the field of view of the protection cameras [ 11 ] .
As mentioned above , the activity of the hot spots is caused by plasma-wall interaction , i.e . a hot spot becomes active when there is sufficient heat flux from the plasma towards the plasma-facing s

First mirror test in JET for ITER: Complete overview after three ILW campaigns
10.1016/j.nme.2019.02.009
Therefore , their performance is crucial for reliable plasma diagnosis and safe operation .
This method gives good depth resolution of a few nm by excellent mass separation [ 17 ] .
There is a slight relative decrease ( 2–3 % ) in near infrared only for the mirror from the cassette entrance ( 0.0 cm ) , while there is even relative improvement by 2–10 % in the ultra violet and a slight positive trend is in the visible range ( < 900 nm ) on other mirrors .
One may assume that this is a potential source of tungsten on the main chamber mirrors .
Truly speaking it is even difficult to find a proper set of techniques to determine conclusively morphology of this very small “ needles ” .
One may very tentatively suggest that they were formed as result of arcing ( parasitic discharge ) between the mirror and the protecting structure ( tube or baffle ) , but there is no chance to prove it .
